In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import matplotlib.pyplot as plt
from qgsw.specs import defaults

In [ ]:
nx = 15*1
ny = 25*1

n_ens = 1
nl = 2

x = torch.linspace(0,2*torch.pi, nx+1, **defaults.get())
y = torch.linspace(0,2*torch.pi, ny+1, **defaults.get())
X,Y = torch.meshgrid(x,y, indexing='ij')

ut = torch.stack([torch.sin(x),torch.sin(2*x)])[None,...]
ub = torch.stack([2*torch.sin(x),2*torch.sin(2*x)])[None,...]
ul = torch.stack([torch.sin(y),torch.sin(2*y)])[None,...]
ur = torch.stack([2*torch.sin(y),2*torch.sin(2*y)])[None,...]

bounds = torch.zeros((nl,nx+1,ny+1),**defaults.get())
bounds[..., :,  0,  :] = ul[0]
bounds[..., :, -1,  :] = ur[0]
bounds[..., :,  :,  0] = ub[0]
bounds[..., :,  :, -1] = ut[0]

In [ ]:
plt.imshow(bounds[0].T.cpu())
plt.colorbar()
plt.show()
plt.imshow(bounds[1].T.cpu())
plt.colorbar()
plt.show()

In [ ]:
ub.shape

In [ ]:
from qgsw.solver.boundary_conditions.base import Boundaries
from qgsw.solver.boundary_conditions.interpolation import BilinearExtendedBoundary

boundaries = Boundaries(ut,ub,ul,ur)
boundary = BilinearExtendedBoundary(boundaries)
u=boundary.compute()
lap_u = boundary.compute_laplacian(dx=x[1]-x[0],dy=y[1]-y[0])

plt.imshow(u[0,0].T.cpu())
plt.colorbar()
# plt.title("Boundary field")
plt.show()
plt.imshow(u[0,1].T.cpu())
plt.colorbar()
# plt.title("Boundary field")
plt.show()

plt.imshow(lap_u[0,0].T.cpu())
plt.colorbar()
# plt.title("Laplacian of the boundary field")
plt.show()
plt.imshow(lap_u[0,1].T.cpu())
plt.colorbar()
# plt.title("Laplacian of the boundary field")
plt.show()

## Tests with random PV values

In [ ]:
from qgsw.models.qg.stretching_matrix import compute_A
from qgsw.physics.coriolis.beta_plane import BetaPlane
from qgsw.solver.pv_inversion import HomogeneousPVInversion, InhomogeneousPVInversion, PVInversion

A = compute_A(torch.tensor([400,1100]),torch.tensor([9.81,0.025]), **defaults.get())

beta_plane = BetaPlane(f0=9.375e-5,beta=1.754e-11)
f0 = beta_plane.f0

q = torch.rand((1,nl,nx,ny),**defaults.get())
dx = x[1]-x[0]
dy = y[1]-y[0]

solver_h = HomogeneousPVInversion(A,f0,dx,dy)
psi_h = solver_h.compute_stream_function(q)

solver_i = InhomogeneousPVInversion(A,f0,dx,dy)
solver_i.set_boundaries(boundaries)

psi_i = solver_i.compute_stream_function(q)

In [ ]:
layer = 0

vmax = q[0,layer].abs().max()

plt.imshow(q[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar()
plt.title("PV")
plt.show()

vmax = psi_h[0,layer].abs().max()
plt.imshow(psi_h[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar()
plt.title("ѱ_h")
plt.show()
plt.imshow(psi_i[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar()
plt.title("ѱ")
plt.show()
vmax = (psi_i[0,layer]-u[0,layer]).abs().max()
plt.imshow((psi_i[0,layer]-u[0,layer]).T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar()
plt.title("ѱ - ѱ_h")
plt.show()

In [ ]:
ut[0,0]-u[0,0,:,-1]

## Tests with PV from previous runs

In [ ]:
from qgsw.configs.space import SpaceConfig
from qgsw.fields.variables.physical import PotentialVorticity
from qgsw.fields.variables.tuples import UVH
from qgsw.models.qg.uvh.core import QG
from qgsw.solver.finite_diff import laplacian_h
from qgsw.spatial.core.discretization import SpaceDiscretization2D
from qgsw.specs import defaults
from qgsw.utils import covphys
from qgsw.utils.units._units import Unit
import torch.nn.functional as F
from qgsw.spatial.core.grid_conversion import points_to_surfaces


nx,ny = 128,256
beta_plane = BetaPlane(
        f0=9.375e-5,
        beta=1.754e-11
    )
space_config = SpaceConfig(
            nx=nx,
            ny=ny,
            unit=Unit.M,
            x_min=-1_280_000,
            x_max=1_280_000,
            y_min=-2_560_000,
            y_max=2_560_000)
space_2d = SpaceDiscretization2D.from_config(space_config)

uvh = UVH.from_file(
    "../output/g5k/double_gyre_qg_long/results_step_876000.pt"
)
model = QG(
    space_2d=space_2d,
    H = torch.tensor([400, 1100, 2600],**defaults.get()),
    g_prime= torch.tensor([9.81, 0.025, 0.0125],**defaults.get()),
    beta_plane=beta_plane
)
model.dt = 3600
model.set_physical_uvh(*uvh)

y = torch.linspace(
    0.5 * model.space.dy,
    model.space.ly - 0.5 * model.space.dy,
    model.space.ny,
    **defaults.get()
).unsqueeze(0)
y0 = 0.5 * model.space.ly
beta_effect = model.beta_plane.beta * (y - y0)

def compute_pv(sf:torch.Tensor)-> torch.Tensor:
    lap_psi = laplacian_h(sf, model.space.dx, model.space.dy)
    stretching = f0**2 * torch.einsum(
        "lm,...mxy->...lxy",
        model.A,
        sf,
    )
    return points_to_surfaces((lap_psi - stretching)) + beta_effect

pv = compute_pv(model.P.compute_p((model.prognostic))[0]/f0)

In [ ]:


x = model.space.omega.xyh.x[0,:,0]
y = model.space.omega.xyh.y[0,0,:]

ut = torch.stack([torch.sin(i*x/model.space.ly*2*torch.pi) for i in range(1,model.space.nl+1)])[None,...]
ub = torch.stack([torch.sin(i*x/model.space.ly*2*torch.pi) for i in range(1,model.space.nl+1)])[None,...]
ul = torch.stack([torch.sin(i*y/model.space.lx*2*torch.pi) for i in range(1,model.space.nl+1)])[None,...]
ur = torch.stack([torch.sin(i*y/model.space.lx*2*torch.pi) for i in range(1,model.space.nl+1)])[None,...]

boundaries = Boundaries(ut,ub,ul,ur)
boundary = BilinearExtendedBoundary(boundaries)
u=boundary.compute()
lap_u = boundary.compute_laplacian(dx=x[1]-x[0],dy=y[1]-y[0])
solver_h = HomogeneousPVInversion(model.A,f0,dx=x[1]-x[0],dy=y[1]-y[0])

psi_h = solver_h.compute_stream_function(pv-beta_effect, ensure_mass_conservation=True)
solver = InhomogeneousPVInversion(model.A,f0,dx=x[1]-x[0],dy=y[1]-y[0])
solver.set_boundaries(boundaries)
psi = solver.compute_stream_function(pv-beta_effect)

layer = 0

fig, axs = plt.subplots(1,5,squeeze=False,figsize=(20,10),constrained_layout=True)

vmax = pv[0,layer].abs().max()
cbar = axs[0,0].imshow(pv[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar(cbar, ax=axs[0,0])
axs[0,0].set_title("PV")

vmax = psi_h[0,layer].abs().max()
cbar = axs[0,1].imshow(psi_h[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar(cbar, ax=axs[0,1])
axs[0,1].set_title("ѱ - 0 at boundary")

vmax = psi[0,layer].abs().max()
cbar = axs[0,2].imshow(psi[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar(cbar, ax=axs[0,2])
axs[0,2].set_title("ѱ")

vmax = (u[0,layer]).abs().max()
cbar = axs[0,3].imshow((u[0,layer]).T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar(cbar, ax=axs[0,3])
axs[0,3].set_title("u")

vmax = (u[0,layer]-psi[0,layer]).abs().max()
cbar = axs[0,4].imshow((psi[0,layer]-u[0,layer]).T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar(cbar, ax=axs[0,4])
axs[0,4].set_title("ѱ - u")

plt.show()

In [ ]:


x = model.space.omega.xyh.x[0,:,0]
y = model.space.omega.xyh.y[0,0,:]

ut = torch.stack([10*psi.abs().mean()*torch.sin(i*x/model.space.ly*2*torch.pi) for i in range(1,model.space.nl+1)])[None,...]
ub = torch.stack([10*psi.abs().mean()*torch.sin(i*x/model.space.ly*2*torch.pi) for i in range(1,model.space.nl+1)])[None,...]
ul = torch.stack([10*psi.abs().mean()*torch.sin(i*y/model.space.lx*2*torch.pi) for i in range(1,model.space.nl+1)])[None,...]
ur = torch.stack([10*psi.abs().mean()*torch.sin(i*y/model.space.lx*2*torch.pi) for i in range(1,model.space.nl+1)])[None,...]

boundaries = Boundaries(ut,ub,ul,ur)
boundary = BilinearExtendedBoundary(boundaries)
u=boundary.compute()
lap_u = boundary.compute_laplacian(dx=x[1]-x[0],dy=y[1]-y[0])
solver_h = HomogeneousPVInversion(model.A,f0,dx=x[1]-x[0],dy=y[1]-y[0])

psi_h = solver_h.compute_stream_function(pv-beta_effect, ensure_mass_conservation=True)
solver = InhomogeneousPVInversion(model.A,f0,dx=x[1]-x[0],dy=y[1]-y[0])
solver.set_boundaries(boundaries)
psi = solver.compute_stream_function(pv-beta_effect)

layer = 0

fig, axs = plt.subplots(1,5,squeeze=False,figsize=(20,10),constrained_layout=True)

vmax = pv[0,layer].abs().max()
cbar = axs[0,0].imshow(pv[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar(cbar, ax=axs[0,0])
axs[0,0].set_title("PV")

vmax = psi_h[0,layer].abs().max()
cbar = axs[0,1].imshow(psi_h[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar(cbar, ax=axs[0,1])
axs[0,1].set_title("ѱ - 0 at boundary")

vmax = psi[0,layer].abs().max()
cbar = axs[0,2].imshow(psi[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar(cbar, ax=axs[0,2])
axs[0,2].set_title("ѱ")

vmax = (u[0,layer]).abs().max()
cbar = axs[0,3].imshow((u[0,layer]).T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar(cbar, ax=axs[0,3])
axs[0,3].set_title("u")

vmax = (u[0,layer]-psi[0,layer]).abs().max()
cbar = axs[0,4].imshow((psi[0,layer]-u[0,layer]).T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar(cbar, ax=axs[0,4])
axs[0,4].set_title("ѱ - u")

plt.show()